In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
filename= "../data/kobe/kobe_bryant_shot_data_refined.csv"
df = pd.read_csv(filename)

In [ ]:
df.head()

In [ ]:
original_df = df.copy()

In [ ]:
# turn categorical variables into dummy variables
categorical_vars = ['combined_shot_type', 'season', 'period']
for var in categorical_vars:
    df = pd.concat([df, pd.get_dummies(df[var], prefix=var)], 1)
    df = df.drop(var, 1)

In [ ]:
df.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, log_loss
from sklearn.cross_validation import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('shot_made_flag', axis=1), 
                                                    df['shot_made_flag'], 
                                                    test_size=0.33, 
                                                    random_state=42)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
model = RandomForestClassifier(random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred_proba = model.predict_proba(X_test)

In [ ]:
confusion_matrix(y_test, y_pred), log_loss(y_test, y_pred_proba[:,1])

In [ ]:
pd.DataFrame({'feature': X_train.columns, 
              'importance': model.feature_importances_}).sort_values('importance', ascending=False).head()

In [ ]:
pred_df = original_df.join(pd.DataFrame(y_pred, columns=['shot_made_pred'], index=X_test.index))

In [ ]:
pred_df = pred_df[~pred_df.shot_made_pred.isnull()]

In [ ]:
pred_df.head()

In [ ]:
pred_df[(pred_df.shot_made_flag != pred_df.shot_made_pred)]

In [ ]:
from sklearn.grid_search import GridSearchCV

param_grid={
    'n_estimators': range(10, 50, 10), 
    'max_depth': range(8, 12),
    'criterion': ['gini', 'entropy'],
}

grid = GridSearchCV(RandomForestClassifier(random_state=42, n_jobs=-1), param_grid=param_grid, scoring='log_loss')

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_score_